# Ableton Live MCP Server - Quickstart

This notebook provides an interactive introduction to the Ableton Live MCP Server.

## Prerequisites

Before running this notebook:

1. **Ableton Live** must be running with a project open
2. **AbletonOSC** remote script must be installed and enabled
3. **ableton-mcp** must be installed (`pip install -e .` from project root)

## 1. Setup and Connection

First, let's import the necessary modules and establish a connection to Ableton Live.

In [ ]:
# Import the container (handles dependency injection)
from ableton_mcp.container import Container
from ableton_mcp.domain.entities import Note

# Initialize the container
container = Container()

# Get the gateway (handles OSC communication)
gateway = container.ableton_gateway()

print("Container initialized!")

In [ ]:
# Connect to Ableton Live
await gateway.connect(
    host="127.0.0.1",
    send_port=11000,
    receive_port=11001,
)

print("Connected to Ableton Live!")

## 2. Getting Song Information

Let's retrieve information about the current Ableton Live project.

In [ ]:
# Get song info
song = await gateway.get_song_info()

print(f"Tempo: {song.tempo} BPM")
print(f"Time Signature: {song.time_signature}")
print(f"Is Playing: {song.is_playing}")

In [ ]:
# Get all tracks
tracks = await gateway.get_tracks()

print(f"Total tracks: {len(tracks)}\n")

for i, track in enumerate(tracks):
    status = []
    if track.is_muted:
        status.append("M")
    if track.is_solo:
        status.append("S")
    if track.is_armed:
        status.append("R")
    status_str = f" [{','.join(status)}]" if status else ""
    
    print(f"[{i}] {track.name}{status_str}")

## 3. Transport Control

Control playback in Ableton Live.

In [ ]:
# Start playback
await gateway.start_playback()
print("Playback started!")

In [ ]:
# Stop playback
await gateway.stop_playback()
print("Playback stopped!")

## 4. Music Theory Services

Use the built-in music theory engine for harmonic analysis.

In [ ]:
# Get the music theory service
music_theory = container.music_theory_service()

# Analyze key from a set of notes
sample_notes = [
    Note(pitch=60, start=0.0, duration=1.0),  # C
    Note(pitch=64, start=1.0, duration=1.0),  # E
    Note(pitch=67, start=2.0, duration=1.0),  # G
    Note(pitch=72, start=3.0, duration=1.0),  # C (octave)
]

keys = await music_theory.analyze_key(sample_notes)

print("Key Analysis Results:")
for key in keys[:3]:
    print(f"  {key.root} {key.mode}: {key.confidence:.1%}")

In [ ]:
# Get chord progression suggestions
progressions = await music_theory.suggest_chord_progressions(
    root="C",
    mode="major",
    genre="pop",
)

print("Chord Progression Suggestions (C major, Pop):")
for i, prog in enumerate(progressions[:5]):
    chords = " -> ".join(prog.chords)
    print(f"  {i+1}. {chords}")

## 5. Track Operations

Create and modify tracks in Ableton Live.

In [ ]:
# Create a new MIDI track
new_track_index = await gateway.create_track(
    track_type="midi",
    name="Notebook Test Track",
    index=-1,  # Append at end
)

print(f"Created track at index: {new_track_index}")

In [ ]:
# Modify track properties
await gateway.set_track_volume(new_track_index, 0.8)
await gateway.set_track_pan(new_track_index, 0.0)

print(f"Track volume set to 80%")
print(f"Track panned to center")

In [ ]:
# Delete the test track (uncomment to run)
# await gateway.delete_track(new_track_index)
# print("Track deleted!")

## 6. Adding MIDI Notes

Add MIDI notes to a clip with optional quantization and scale filtering.

In [ ]:
# Define some notes (C major arpeggio)
notes = [
    Note(pitch=60, start=0.0, duration=0.5, velocity=100),  # C4
    Note(pitch=64, start=0.5, duration=0.5, velocity=90),   # E4
    Note(pitch=67, start=1.0, duration=0.5, velocity=90),   # G4
    Note(pitch=72, start=1.5, duration=0.5, velocity=100),  # C5
]

print("Notes defined:")
for note in notes:
    print(f"  Pitch {note.pitch} at beat {note.start}")

In [ ]:
# Add notes to the first track, first clip
# (Make sure there's a MIDI clip in that slot!)

track_index = 0
clip_index = 0

try:
    await gateway.add_notes(
        track_id=track_index,
        clip_id=clip_index,
        notes=notes,
        quantize=True,
    )
    print(f"Added {len(notes)} notes to Track {track_index}, Clip {clip_index}")
except Exception as e:
    print(f"Could not add notes: {e}")
    print("Make sure there's a MIDI clip in the target slot.")

## 7. Cleanup

Disconnect from Ableton Live when done.

In [ ]:
# Disconnect
await gateway.disconnect()
print("Disconnected from Ableton Live.")

## Next Steps

Now that you've learned the basics, explore more:

- **[examples/](.)** - More example scripts
- **[docs/](../docs/)** - Full documentation
- **[README.md](../README.md)** - API reference

Happy music making!